# **TPS FEB2022**

### Interesting tabular data

### Day1: I am trying visualize what kind of data and built the base model

STEPS:

1. Import necessary data
2. Load the data, shape
3. Stastical Analysis and Identify Null Data
4. KFOLD
5. Feature Separation
6. Apply preprocessing only target data(Label encoder--> 10 class of data)
7. Build the model(KNNClassifier|RobustScaler|Predict)


## **Import necessary Library**

In [ ]:
#Import necessary Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time

# Preprocessing
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('float_format','{:f}'.format)

## **Load,Shape,Describe**

In [ ]:
#DataLoad, Shape, Describe

#import the data and shape
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")
sample=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
print(train.shape,test.shape,sample.shape)
train.describe().transpose()

In [ ]:
train.columns

In [ ]:
print(f'Total number of Train data: {train.shape[0]}')
print(f'\033[92mTotal number of Test data: {test.shape[0]}')
print(f'\033[96mNumber of Train_null identify: {sum(train.isna().sum())}')
print(f'\033[96mNumber of Test_null identify: {sum(test.isna().sum())}')


In [ ]:
#insert the kfold columns
train['kfold'] = -1
#distributing the data
kfold=KFold(n_splits=5,random_state=42)
for fold, (tr_i,va_i) in enumerate(kfold.split(X=train)):
    train.loc[va_i,'kfold'] = fold
    
print(train.kfold.value_counts())
train.to_csv("folds_5.csv",index=False)
print("successfully folds")

## **Feature separation**

In [ ]:
#feature separation
df= pd.read_csv("./folds_5.csv")
#train.head()
#features taken to train
TARGET = 'target'
features = [f for f in df.columns if f not in("row_id","kfold",TARGET)]
test= test[features]


## **Preprocessing the Target**

In [ ]:
#preprocessing
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,RobustScaler
LE = LabelEncoder()
df[TARGET] = LE.fit_transform(train[TARGET])
#see the target
df.head() 

In [ ]:
df.target.unique()

## **Build the model**

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBRegressor,XGBClassifier
#initialize the model
prediction = []
score = []

for fold in range (5):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
        
    #Model hyperparameter of XGboostRegressor
    xgb_params = {
        'learning_rate': 0.001235,
        'subsample': 0.95312,
        'colsample_bytree': 0.1107,
        'max_depth': 3,
        'booster': 'gbtree', 
        'reg_lambda': 66.156,
        'reg_alpha': 14.68267919457715,
        'random_state':42,
        'n_estimators':15000,
        'objective': 'multi:softmax',
        'eval_metric':'mlogloss'
    }
    
    model= XGBClassifier(**xgb_params,
                       gpu_id=0,
                       tree_method='gpu_hist',
                       predictor='gpu_predictor')
    model.fit(xtrain,ytrain,early_stopping_rounds=100,eval_set=[(xvalid,yvalid)],verbose=False)
    
    preds_valid = model.predict(xvalid)
    
    test_pre = model.predict(xtest)
    prediction.append(test_pre)
    
    acc= accuracy_score(yvalid,preds_valid)
    #Score 
    score.append(acc)
    print(f"fold|split:{fold},ACC:{acc}")
    
print(np.mean(score),np.std(score))


## **Output**

In [ ]:
#reconfigure of split data
final_predict = LE.inverse_transform(np.squeeze(mode(np.column_stack(prediction),axis=1)[0]).astype('int'))
print(final_predict)
sample.target = final_predict
sample.to_csv("submission.csv",index=False)
print("Final achieve to send xgbclassifier output data")


In [ ]:
sample.head()

## **Thankyou for Visiting!** 

Suggest Any new ways and tips